In [1]:
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
from google.cloud import storage
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
project_id = "looker-assignment-113356033"
dataset_id = "final_project_dataset"

In [3]:
crendentials = service_account.Credentials.from_service_account_file(r"C:\nccu\workspace\dataModel_finalProjrct\looker-assignment-113356033-4959799503ac.json")

In [4]:
bigquery_client = bigquery.Client(project=project_id, credentials=crendentials)
storage_client = storage.Client(credentials=crendentials, project=project_id)

In [5]:
Bonk_token_balances_query = """
SELECT
    *
FROM `looker-assignment-113356033.transfer_all.Bonk_token_balances`
ORDER BY year_month
"""

Floki_token_balances_query = """
SELECT
    *
FROM `looker-assignment-113356033.transfer_all.Bonk_token_balances`
ORDER BY year_month
"""

Mog_token_balances_query = """
SELECT
    *
FROM `looker-assignment-113356033.transfer_all.Bonk_token_balances`
ORDER BY year_month
"""

Pepe_token_balances_query = """
SELECT
    *
FROM `looker-assignment-113356033.transfer_all.Bonk_token_balances`
ORDER BY year_month
"""

Shib_token_balances_query = """
SELECT
    *
FROM `looker-assignment-113356033.transfer_all.Bonk_token_balances`
ORDER BY year_month
"""

Bonk_token_balances = bigquery_client.query(Bonk_token_balances_query).to_dataframe() 

C:\Users\wise\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [ ]:
# Queries for each token holder dataset
queries = {
    "Bonk": """
        SELECT
            *
        FROM `looker-assignment-113356033.final_project_dataset.Bonk_compare_holders`
    """,
    "FLOKI": """
        SELECT
            *
        FROM `looker-assignment-113356033.final_project_dataset.FLOKI_compare_holders`
    """,
    "Mog": """
        SELECT
            *
        FROM `looker-assignment-113356033.final_project_dataset.Mog_compare_holders`
    """,
    "Pepe": """
        SELECT
            *
        FROM `looker-assignment-113356033.final_project_dataset.Pepe_compare_holders`
    """,
    "Shib": """
        SELECT
            *
        FROM `looker-assignment-113356033.final_project_dataset.Shib_compare_holders`
    """
}

try:
    # 處理每個代幣
    for token_name, query in queries.items():
        try:
            # 執行 BigQuery 查詢並載入 DataFrame
            df = bigquery_client.query(query).to_dataframe()

            # 檢查是否有 'Address' 和 'Balance' 欄位
            if 'Address' in df.columns and 'Balance' in df.columns:
                # 根據 Address 聚合，對 Balance 進行加總，保留所有其他欄位
                aggregated_df = df.groupby('Address', as_index=False).agg({
                    col: 'first' if col not in ['Balance'] else 'sum'
                    for col in df.columns
                })

                # 篩選 Balance > 0 的行
                aggregated_df = aggregated_df[aggregated_df['Balance'] > 0]

                # 排序結果（可選）
                aggregated_df = aggregated_df.sort_values(by='Balance', ascending=False).reset_index(drop=True)

                # 保存結果到 CSV 文件
                output_file = f'processed_{token_name}_compare_holders.csv'
                aggregated_df.to_csv(output_file, index=False)

                print(f"[INFO] {token_name} 資料已處理完成，結果保存到 {output_file}")

            else:
                print(f"[WARNING] {token_name} 資料缺少 'Address' 或 'Balance' 欄位，已跳過。")

        except Exception as e:
            print(f"[ERROR] 無法處理 {token_name} 資料: {e}")

    print("[INFO] 所有文件已處理完成。")

except Exception as e:
    print(f"[ERROR] 發生錯誤: {e}")

[INFO] 文件 Floki_compare_holders.csv 已處理完成，結果保存到 transform_data\processed_Floki_compare_holders.csv
[INFO] 文件 Bonk_compare_holders.csv 已處理完成，結果保存到 transform_data\processed_Bonk_compare_holders.csv
[INFO] 文件 Mog_compare_holders.csv 已處理完成，結果保存到 transform_data\processed_Mog_compare_holders.csv
[INFO] 文件 Shib_compare_holders.csv 已處理完成，結果保存到 transform_data\processed_Shib_compare_holders.csv
[INFO] 文件 Pepe_compare_holders.csv 已處理完成，結果保存到 transform_data\processed_Pepe_compare_holders.csv
[INFO] 所有文件已處理完成。


In [ ]:
# 目錄包含所有文件
directory_path = 'transform_data'

# 文件名稱列表
file_names = [
    'processed_Floki_compare_holders.csv',
    'processed_Bonk_compare_holders.csv',
    'processed_Mog_compare_holders.csv',
    'processed_Shib_compare_holders.csv',
    'processed_Pepe_compare_holders.csv'
]
for file_name in file_names:
    file_path = os.path.join(directory_path, file_name)

    print(f"[INFO] 文件 {file_name} 已處理完成")

    # 讀取 CSV 文件
    df = pd.read_csv(file_path)
    token_columns = ['Hold FLOKI', 'Hold PEPE', 'Hold SHIB', 'Hold MOG', 'Hold BONK']
    column_sums = (df[token_columns]).sum()
    print(column_sums)

[INFO] 文件 processed_Floki_compare_holders.csv 已處理完成
Hold FLOKI    91507
Hold PEPE     17503
Hold SHIB     22425
Hold MOG       3439
Hold BONK      1816
dtype: int64
[INFO] 文件 processed_Bonk_compare_holders.csv 已處理完成
Hold FLOKI     1822
Hold PEPE      5052
Hold SHIB      3512
Hold MOG       1192
Hold BONK     14991
dtype: int64
[INFO] 文件 processed_Mog_compare_holders.csv 已處理完成
Hold FLOKI     3981
Hold PEPE     16605
Hold SHIB      5992
Hold MOG      50959
Hold BONK      1353
dtype: int64
[INFO] 文件 processed_Shib_compare_holders.csv 已處理完成
Hold FLOKI      21768
Hold PEPE       71069
Hold SHIB     1594515
Hold MOG         5022
Hold BONK        3354
dtype: int64
[INFO] 文件 processed_Pepe_compare_holders.csv 已處理完成
Hold FLOKI     19006
Hold PEPE     399324
Hold SHIB      75003
Hold MOG       15518
Hold BONK       5295
dtype: int64
